In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Google-Stock-Prediction-V3'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

     symbol                       date    close      high        low     open  \
0      GOOG  2016-06-14 00:00:00+00:00   718.27   722.470   713.1200   716.48   
1      GOOG  2016-06-15 00:00:00+00:00   718.92   722.980   717.3100   719.00   
2      GOOG  2016-06-16 00:00:00+00:00   710.36   716.650   703.2600   714.91   
3      GOOG  2016-06-17 00:00:00+00:00   691.72   708.820   688.4515   708.65   
4      GOOG  2016-06-20 00:00:00+00:00   693.71   702.480   693.4100   698.77   
...     ...                        ...      ...       ...        ...      ...   
1253   GOOG  2021-06-07 00:00:00+00:00  2466.09  2468.000  2441.0725  2451.32   
1254   GOOG  2021-06-08 00:00:00+00:00  2482.85  2494.495  2468.2400  2479.90   
1255   GOOG  2021-06-09 00:00:00+00:00  2491.40  2505.000  2487.3300  2499.50   
1256   GOOG  2021-06-10 00:00:00+00:00  2521.60  2523.260  2494.0000  2494.01   
1257   GOOG  2021-06-11 00:00:00+00:00  2513.93  2526.990  2498.2900  2524.92   

       volume  adjClose   a

In [4]:
data = data['close']

In [5]:
data

0        718.27
1        718.92
2        710.36
3        691.72
4        693.71
         ...   
1253    2466.09
1254    2482.85
1255    2491.40
1256    2521.60
1257    2513.93
Name: close, Length: 1258, dtype: float64

In [6]:
data = data.tolist()

In [7]:
data

[718.27,
 718.92,
 710.36,
 691.72,
 693.71,
 695.94,
 697.46,
 701.87,
 675.22,
 668.26,
 680.04,
 684.11,
 692.1,
 699.21,
 694.49,
 697.77,
 695.36,
 705.63,
 715.09,
 720.64,
 716.98,
 720.95,
 719.85,
 733.78,
 736.96,
 741.19,
 738.63,
 742.74,
 739.77,
 738.42,
 741.77,
 745.91,
 768.79,
 772.88,
 771.07,
 773.18,
 771.61,
 782.22,
 781.76,
 784.26,
 784.68,
 784.85,
 783.22,
 782.44,
 777.14,
 779.91,
 777.5,
 775.42,
 772.15,
 772.08,
 769.64,
 769.41,
 769.54,
 772.15,
 769.09,
 767.05,
 768.78,
 771.46,
 780.08,
 780.35,
 775.32,
 759.66,
 769.02,
 759.69,
 762.49,
 771.76,
 768.88,
 765.7,
 771.41,
 776.22,
 787.21,
 786.9,
 774.21,
 783.01,
 781.56,
 775.01,
 777.29,
 772.56,
 776.43,
 776.47,
 776.86,
 775.08,
 785.94,
 783.07,
 786.14,
 778.19,
 778.53,
 779.96,
 795.26,
 801.56,
 796.97,
 799.37,
 813.11,
 807.67,
 799.07,
 795.35,
 795.37,
 784.54,
 783.61,
 768.7,
 762.13,
 762.02,
 782.52,
 790.51,
 785.31,
 762.56,
 754.02,
 736.08,
 758.49,
 764.48,
 771.23,
 760.5

In [8]:
data = torch.from_numpy(np.array(data))

In [9]:
data

tensor([ 718.2700,  718.9200,  710.3600,  ..., 2491.4000, 2521.6000,
        2513.9300], dtype=torch.float64)

In [10]:
data.view(1,-1)

tensor([[ 718.2700,  718.9200,  710.3600,  ..., 2491.4000, 2521.6000,
         2513.9300]], dtype=torch.float64)

In [11]:
data = data.view(1,-1)

In [12]:
data = data.to(device).float()

In [13]:
data

tensor([[ 718.2700,  718.9200,  710.3600,  ..., 2491.3999, 2521.6001,
         2513.9299]], device='cuda:0')

In [14]:
data_input = data[:,:-1]
data_target = data[:,1:]

In [15]:
data_input,data_target

(tensor([[ 718.2700,  718.9200,  710.3600,  ..., 2482.8501, 2491.3999,
          2521.6001]], device='cuda:0'),
 tensor([[ 718.9200,  710.3600,  691.7200,  ..., 2491.3999, 2521.6001,
          2513.9299]], device='cuda:0'))

In [16]:
data

tensor([[ 718.2700,  718.9200,  710.3600,  ..., 2491.3999, 2521.6001,
         2513.9299]], device='cuda:0')

In [17]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.ouput = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden),torch.zeros(batch_size,self.hidden)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden),torch.zeros(batch_size,self.hidden)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.ouput(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.ouput(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [18]:
model = Model().to(device)
criterion = MSELoss()
optimizer = LBFGS(model.parameters(),lr=0.8)
epochs = 100

In [19]:
wandb.init(projet=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_input.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [20]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_input.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [21]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.ouput = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.ouput(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.ouput(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [22]:
model = Model().to(device)
criterion = MSELoss()
optimizer = LBFGS(model.parameters(),lr=0.8)
epochs = 100

In [23]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_input.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()